In [1]:
import asyncio

import polars as pl

from humbldata.core.standard_models.abstract.query_params import (
    QueryParams,  # Base for command_params
)
from humbldata.core.standard_models.toolbox import (
    ToolboxQueryParams,  # General context params
)
# Core imports (adjust paths if running from a different location)
from humbldata.core.utils.env import Env
from humbldata.core.utils.openbb_api_client import OpenBBAPIClient



In [2]:
    from typing import Optional, Literal
    from pydantic import BaseModel, Field

    class EquityPriceHistoricalQueryParams(BaseModel):
        symbol: str | list[str]
        start_date: Optional[str] = None
        end_date: Optional[str] = None
        interval: Optional[str] = Field(default='1d')
        # ... other fields as defined before, matching your API
        provider: Optional[str] = Field(default="yfinance") # Ensure this matches your API
        chart: Optional[bool] = Field(default=False)
        # Add all relevant fields from your example API call
        adjustment: Optional[Literal['splits_only', 'dividends_only', 'all']] = Field(default='splits_only')
        extended_hours: Optional[bool] = Field(default=False)
        use_cache: Optional[bool] = Field(default=True)
        timezone: Optional[str] = Field(default="America/New_York")
        source: Optional[str] = Field(default="realtime")
        sort: Optional[Literal['asc', 'desc']] = Field(default='asc')
        limit: Optional[int] = Field(default=49999)
        include_actions: Optional[bool] = Field(default=True)

In [3]:


# A dummy CommandParams class for this example, mirroring a real command's QueryParams
class ExampleCommandParams(QueryParams):
    symbol: str
    start_date: str
    end_date: str
    # other fields specific to a command that might use this API call


In [4]:
import nest_asyncio
nest_asyncio.apply()

async def main():
    # 1. Initialize Environment
    env = Env()
    print(f"Running in environment: {env.ENVIRONMENT}")
    base_url_to_check = await OpenBBAPIClient()._get_base_url() # For checking
    print(f"API Base URL: {base_url_to_check}")


    # 2. Initialize OpenBBAPIClient
    api_client = OpenBBAPIClient()

    # 3. Prepare Context and Command Parameters for HumblObject
    # These would normally come from your application's context/command structure
    context_params = ToolboxQueryParams(
        symbols=["AAPL"],
        provider="yfinance", # Indicate the source
        # ... other context fields if necessary
    )

    # These are the parameters specific to the command that initiated this data request.
    # For example, if HumblCompassBacktest was calling this, these would be its params.
    # Here, we create a simple version.
    example_command_params = ExampleCommandParams(
        symbol="AAPL",
        start_date="2023-01-01",
        end_date="2023-01-10"
    )


    # 4. Prepare API-specific query parameters
    # These parameters are specific to the '/equity/price/historical' API endpoint
    equity_historical_api_params = EquityPriceHistoricalQueryParams(
        symbol=context_params.symbols,
        start_date=example_command_params.start_date,
        end_date=example_command_params.end_date,
        provider=context_params.provider # Assuming your API uses a provider param
    )

    # 5. Fetch data
    print(f"Fetching data for {equity_historical_api_params.symbol}...")
    humbl_object_response = await api_client.fetch_data(
            obb_path="equity.price.historical", # OpenBB-like path
            api_query_params=equity_historical_api_params, # API specific params
        )
    return humbl_object_response


if __name__ == "__main__":
    result = asyncio.run(main())
    result


INFO: OpenBBAPIClient || START: fetch_data (async)
INFO: OpenBBAPIClient || Prepared request for: http://127.0.0.1:6900/api/v1/equity/price/historical?symbol=AAPL&start_date=2023-01-01&end_date=2023-01-10&interval=1d&provider=yfinance&chart=false&adjustment=splits_only&extended_hours=false&use_cache=true&timezone=America/New_York&source=realtime&sort=asc&limit=49999&include_actions=true
INFO: OpenBBAPIClient || Fetching data from: http://127.0.0.1:6900/api/v1/equity/price/historical?symbol=AAPL&start_date=2023-01-01&end_date=2023-01-10&interval=1d&provider=yfinance&chart=false&adjustment=splits_only&extended_hours=false&use_cache=true&timezone=America/New_York&source=realtime&sort=asc&limit=49999&include_actions=true


Running in environment: development
API Base URL: http://127.0.0.1:6900/api/v1
Fetching data for ['AAPL']...


/Users/jjfantini/github/humblFINANCE-org/humblDATA/src/humbldata/core/utils/openbb_api_client.py:310: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  if "date" in results_lf.schema:
/Users/jjfantini/github/humblFINANCE-org/humblDATA/src/humbldata/core/utils/openbb_api_client.py:311: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  dtype = results_lf.schema["date"]
INFO: OpenBBAPIClient || END: fetch_data (async) - Total time: 1.3636s


In [5]:
result


HumblObject

id: 06825667-7b3c-7686-8000-2988b8940a1f
results: b'DSL_VERSION\x01\x00\x00\x00\x81\xa6HStack\x83\xa5input\x81\xa2IR\x82\xa3...
equity_data: None
provider: yfinance
warnings: [{'category': 'HumblDataWarning', 'message': 'Start date adjusted to 2024...
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'}, 's...
context_params: None
command_params: None

In [6]:
result.to_polars()

date,open,high,low,close,volume
date,f64,f64,f64,f64,i64
2023-01-03,130.279999,130.899994,124.169998,125.07,112117500
2023-01-04,126.889999,128.660004,125.080002,126.360001,89113600
2023-01-05,127.129997,127.769997,124.760002,125.019997,80962700
2023-01-06,126.010002,130.289993,124.889999,129.619995,87754700
2023-01-09,130.470001,133.410004,129.889999,130.149994,70790800
2023-01-10,130.259995,131.259995,128.119995,130.729996,63896200


In [ ]:
import polars as pl
lf = pl.LazyFrame(
    {
        "date": [1, 2, 3],
        "bar": [6.0, 7.0, 8.0],
        "ham": ["a", "b", "c"],
    }
)
lf.collect_schema()

TypeError: list indices must be integers or slices, not str